# Lab09. Phenology Modeling 
Case studies of modeling plant phenology with corn leaf development and cherry blossoms

In [ ]:
using Cropbox

In [ ]:
Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
using CSV
using DataFrames
using Dates
using TimeZones

## Growing Degree Days

### Ex 9.1

The maize data we used in the lab unit 3 for calibrating a sigmoidal growth curve originally came from a field experiment took place in Beltsville, Maryland at the research farm of USDA-ARS Beltsville Agricultural Research Center (BARC) in 2002. The data sets include daily weather, weather with every 5 min intervals, biomass by parts, leaf area growth per plant, and the number of leaves determined by counting leaf tips weekly. Planting date was 18 May 2002.

- Eq 9.1 & 9.2

$$
\begin{align}
\mathrm{GDD}(T) &= \max \{ 0, \min \{ T, T_{opt} \} - T_b \} \\
\mathrm{cGDD} &= \sum_i^n \mathrm{GDD}(T_i) \\
\end{align}
$$

#### Ex 9.1.1

Using the daily weather data, determine the growing degree days (GDD) ($T_b$ = 8.0 °C and $T_{opt}$ = 32.0 °C) accrued for each day.

In [ ]:
@system GrowingDegreeDay begin
    T:  temperature         ~ preserve(parameter, u"°C")
    Tb: base_temperature    ~ preserve(parameter, u"°C")
    To: optimal_temperature ~ preserve(parameter, u"°C")

    GD(T, Tb, To): growing_degree => begin
        min(T, To) - Tb
    end ~ track(min = 0, u"K")

    GDD(GD): growing_degree_day => begin
        GD / 1u"d"
    end ~ track(u"K/d")

    cGDD(GDD): cumulative_growing_degree_day ~ accumulate(u"K")
end

In [ ]:
@system GDDController(GrowingDegreeDay, Controller)

In [ ]:
gdd_config = @config (
    Clock => (;
        step = 1u"d",
    ),
    GrowingDegreeDay => (;
        Tb = 8.0,
        To = 32.0,
    ),
)

In [ ]:
visualize(GDDController, :T, :GDD;
    config = gdd_config,
    xstep = :0 => :T => 0:40,
    kind = :line,
)

In [ ]:
manipulate(GDDController, :T, :GDD;
    config = Clock => :step => 1u"d",
    parameters = GrowingDegreeDay => (;
        Tb = 0:0.1:10,
        To = 1:40,
    ),
    xstep = :0 => :T => -5:45,
    ylim = (0, 40),
    kind = :line,
)

In [ ]:
corn_weather = CSV.read("corn_weather.csv", DataFrame) |> unitfy

In [ ]:
@system Temperature begin
    calendar(context)             ~      ::Calendar
    t(calendar.date): date        ~ track::date

    data                          ~ provide(parameter, index = :date, init = t)
    T:                temperature ~ drive(from = data, by = :Tavg, u"°C")
end

`Temperature` is a mix-in system for loading up temperature variable from an external data source. It relies on variables with kind of `provide` and `drive`. For handling time variable in calendar date format (YYYY/MM/DD), we also deploy a `Calendar` system included with Cropbox.

- `provide` *provides* a data frame with given index (`index`) starting from an initial value (`init`). Since we're going to provide a data frame using a configuration, the variable is tagged `parameter`.

- `drive` makes a *driving* variable from a data source (`from`) with a given column name (`by`). The data source is often supplied by `provide`.

- `Calendar` is like `Clock` embedded in `Context` system that provides `time` and `step` variables, but in the type of `ZonedDateTime` (`datetime` in Cropbox). A full specification looks like below.

```julia
@system Calendar begin
    init ~ preserve::datetime(extern, parameter)
    last => nothing ~ preserve::datetime(extern, parameter, optional)
    time(t0=init, t=context.clock.time) => t0 + convert(Cropbox.Dates.Second, t) ~ track::datetime
    date(time) => Cropbox.Dates.Date(time) ~ track::date
    step(context.clock.step) ~ preserve(u"hr")
    stop(time, last) => begin
        isnothing(last) ? false : (time >= last)
    end ~ flag
    count(init, last, step) => begin
        if isnothing(last)
            nothing
        else
            # number of update!() required to reach `last` time
            (last - init) / step
        end
    end ~ preserve::int(round, optional)
end
```

In [ ]:
corn_config = @config (
    Calendar => (;
        init = ZonedDateTime(2002, 5, 15, tz"America/New_York"),
        last = ZonedDateTime(2002, 9, 30, tz"America/New_York"),
    ),
    Clock => (;
        step = 1u"d",
    ),
    Temperature => (;
        data = corn_weather,
    ),
)

`Calendar` system embedded in `Temperature` above accepts `init` and `last` parameters in the type of `ZonedDateTime` for representing timestamps with proper time zone support. We need time zone because the default time resolution in Cropbox is by an hour and things can become tricky when it comes to handling daylight savings. To simplify our exercises, we will use daily time step for the most part of simulations here. `Calendar` also provides `stop` variable that taps on the interval between `init` and `last` to inform `simulate()` when simulation should be done.

In [ ]:
@system GDDCalculator(GrowingDegreeDay, Temperature, Controller)

In [ ]:
gddc_config = @config (
    corn_config,
    gdd_config,
)

In [ ]:
simulate(GDDCalculator;
    config = gddc_config,
    stop = "calendar.stop",
    index = :t,
    target = [:GDD, :cGDD],
)

In [ ]:
visualize(GDDCalculator, :t, :T;
    config = gddc_config,
    stop = "calendar.stop",
    kind = :line
)

In [ ]:
visualize(GDDCalculator, :t, :GDD;
    config = gddc_config,
    stop = "calendar.stop",
    kind = :line,
)

#### Ex 9.1.2

Determine cumulative GDD (cGDD) since planting until harvest on 30 September 2002.

In [ ]:
visualize(GDDCalculator, :t, :cGDD;
    config = gddc_config,
    stop = "calendar.stop",
    kind = :line,
)

#### Ex 9.1.3

Determine leaf appearance rate for up to 17 leaves based on phyllochron of 43.0.

In [ ]:
@system LeafAppearanceGD(GrowingDegreeDay, Temperature, Controller) begin
    N: leaves_total ~ preserve(parameter)
    p: phyllochron  ~ preserve(parameter, u"K")
    
    n(cGDD, p): leaves_appeared => begin
        cGDD / p
    end ~ track::int(round = :floor)

    m(n, N): match             => (n >= N) ~ flag
    stop(m, s = calendar.stop) => (m || s) ~ flag
end

`flag` is a kind of variable similar to `track` associated with `Bool` (ture or false) type. Additionally, it allows composition with logical operators in supported tags like `when` for some variables like `track` and `accumulate`.

- `match` is a flag variable indicating a certain threshold of thermal accumulation is fulfilled.
- `stop` is a flag variable to determine if the simulation needs to be terminated when a successful match is found or when it reaches the `last` day of calendar.

In [ ]:
lagd_config = @config (
    corn_config,
    gdd_config,
    LeafAppearanceGD => (;
        N = 18,
        p = 43.0,
    ),
)

In [ ]:
lagd_result = simulate(LeafAppearanceGD;
    config = lagd_config,
    stop = :stop,
    index = :t,
    target = [:T, :GDD, :cGDD, :p, :n, :match, :stop],
)

In [ ]:
last(lagd_result, 10)

In [ ]:
visualize(LeafAppearanceGD, :t, [:N, :n];
    config = lagd_config,
    stop = :stop,
    kind = :line,
)

#### Ex 9.1.4

Plot the predicted and observed leaf appearance data over days after planting (DAP) in the same figure.

In [ ]:
corn_obs = CSV.read("corn_sample.csv", DataFrame) |> unitfy

In [ ]:
p = visualize(corn_obs, :date, :TotLfNo;
    xlim = (Date(2002, 5), Date(2002, 9)),
    ylim = (0, 20),
    name = "Obs",
)
visualize!(p, LeafAppearanceGD, :t, :n;
    config = lagd_config,
    stop = :stop,
    kind = :line,
    name = "Model",
)

In [ ]:
p = visualize(corn_obs, :date, :VSLfNo;
    xlim = (Date(2002, 5), Date(2002, 9)),
    ylim = (0, 20),
    name = "Obs",
)
visualize!(p, LeafAppearanceGD, :t, :n;
    config = lagd_config,
    stop = :stop,
    kind = :line,
    name = "Model",
)

#### Ex 9.1.5

Determine RMSE and EF, and interpret and discuss your findings.

In [ ]:
evaluate(LeafAppearanceGD, corn_obs;
    config = lagd_config,
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :rmse,
)

In [ ]:
evaluate(LeafAppearanceGD, corn_obs;
    config = lagd_config,
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :ef,
)

Calibrate

In [ ]:
lagd_calib_config = calibrate(LeafAppearanceGD, corn_obs;
    config = lagd_config,
    index = :date => :t,
    target = :LfNo => :n,
    parameters = LeafAppearanceGD => (;
        p = (45.0, 55.0),
    ),
    stop = :stop,
    optim = (;
        MaxSteps = 5000,
    ),
    metric = :rmse,
)

In [ ]:
p = visualize(corn_obs, :date, :TotLfNo;
    xlim = (Date(2002, 5), Date(2002, 9)),
    ylim = (0, 20),
    name = "Obs",
)
visualize!(p, LeafAppearanceGD, :t, :n;
    config = (lagd_config, lagd_calib_config),
    stop = :stop,
    kind = :line,
    name = "Model",
)

In [ ]:
evaluate(LeafAppearanceGD, corn_obs;
    config = (lagd_config, lagd_calib_config),
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :rmse,
)

In [ ]:
evaluate(LeafAppearanceGD, corn_obs;
    config = (lagd_config, lagd_calib_config),
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :ef,
)

## Beta Distribution Function

### Ex 9.2

Using the same data as above, answer the following questions based on the leaf tip appearance rate predicted by the beta function.

- Eq 9.3

$$
r(T) = R_{max} \left( \frac{T_{ceil} - T}{T_{ceil} - T_{opt}} \right) \left( \frac{T - T_{min}}{T_{opt} - T_{min}} \right) ^ {\frac{T_{opt} - T_{min}}{T_{ceil} - T_{opt}}}
$$

Here we explicitly use $T_{min}$ to make sure we have a proper unit (`K` instead of `°C`) from [calculation bewteen temperature variables](https://painterqubits.github.io/Unitful.jl/stable/temperature/).

#### Ex 9.2.1

Create a system to predict the leaf appearance rate based on beta function with the parameter values found in Yan and Hunt (1999): $R_{max}$ = 0.58 $\mathrm{d^{-1}}$, $T_{opt}$ = 31.4 °C, and $T_{ceil}$ = 41.0 °C.

In [ ]:
@system BetaFunc begin
    T:  temperature              ~ preserve(parameter, u"°C")
    Tn: minimum_temperature => 0 ~ preserve(parameter, u"°C")
    To: optimal_temperature      ~ preserve(parameter, u"°C")
    Tc: ceiling_temperature      ~ preserve(parameter, u"°C")
    Rx: maximum_rate        => 1 ~ preserve(parameter, u"d^-1")

    r(T, Tn, To, Tc, Rx): rate => begin
        Rx * (Tc - T) / (Tc - To) * ((T - Tn) / (To - Tn))^((To - Tn) / (Tc - To))
    end ~ track(min = 0, max = Rx, u"d^-1")

    R(r): cumulative_rate ~ accumulate
end

In [ ]:
@system BFController(BetaFunc, Controller)

In [ ]:
bf_config = @config (
    BetaFunc => (;
        Rx = 0.58,
        To = 31.4,
        Tc = 41.0,
    ),
)

In [ ]:
visualize(BFController, :T, :r;
    config = bf_config,
    xstep = :0 => :T => 0:40,
    kind = :line,
)

In [ ]:
manipulate(BFController, :T, :r;
    parameters = BetaFunc => (;
        Rx = 0:0.1:1,
        To = 0:0.1:40,
        Tc = 0:0.1:60,
    ),
    xstep = :0 => :T => 0:50,
    ylim = (0, 1),
    kind = :line,
)

#### Ex 9.2.2

Using daily weather data, simulate the leaf tip appearance up to 17 leaves throughout the experiment based on the beta function model.

In [ ]:
@system BFCalculator(BetaFunc, Temperature, Controller)

In [ ]:
bfc_config = @config (
    corn_config,
    bf_config,
)

In [ ]:
simulate(BFCalculator;
    config = bfc_config,
    stop = "calendar.stop",
    index = :t,
    target = [:r, :R],
)

In [ ]:
visualize(BFCalculator, :t, :r;
    config = bfc_config,
    stop = "calendar.stop",
    kind = :line,
)

In [ ]:
visualize(BFCalculator, :t, :R;
    config = bfc_config,
    stop = "calendar.stop",
    kind = :line,
)

In [ ]:
@system LeafAppearanceBF(BetaFunc, Temperature, Controller) begin
    N: leaves_total       ~ preserve::int(parameter)
    
    n(R): leaves_appeared ~ track::int(round = :floor)
    
    match(n, N)                        => (n >= N) ~ flag
    stop(m = match, s = calendar.stop) => (m || s) ~ flag
end

In [ ]:
labf_config = @config (
    corn_config,
    bf_config,
    LeafAppearanceBF => (;
        N = 18,
    ),
)

In [ ]:
labf_result = simulate(LeafAppearanceBF;
    config = labf_config,
    stop = :stop,
    index = :t,
    target = [:T, :r, :R, :n, :match, :stop],
)

In [ ]:
last(labf_result, 10)

In [ ]:
visualize(LeafAppearanceBF, :t, [:N, :n];
    config = labf_config,
    stop = :stop,
    kind = :line,
)

In [ ]:
p = visualize(corn_obs, :date, :TotLfNo;
    xlim = (Date(2002, 5), Date(2002, 9)),
    ylim = (0, 20),
    name = "Obs",
)
visualize!(p, LeafAppearanceBF, :t, :n;
    config = labf_config,
    stop = :stop,
    kind = :line,
    name = "Model",
)

In [ ]:
p = visualize(corn_obs, :date, :VSLfNo;
    xlim = (Date(2002, 5), Date(2002, 9)),
    ylim = (0, 20),
    name = "Obs",
)
visualize!(p, LeafAppearanceBF, :t, :n;
    config = labf_config,
    stop = :stop,
    kind = :line,
    name = "Model",
)

#### Ex 9.2.4

Determine RMSE and EF, and interpret and discuss your findings.

In [ ]:
evaluate(LeafAppearanceBF, corn_obs;
    config = labf_config,
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :rmse,
)

In [ ]:
evaluate(LeafAppearanceBF, corn_obs;
    config = labf_config,
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :ef,
)

Calibrate

In [ ]:
labf_calib_config = calibrate(LeafAppearanceBF, corn_obs;
    config = labf_config,
    index = :date => :t,
    target = :LffNo => :n,
    parameters = LeafAppearanceBF => (;
        Rx= (0.4, 0.7),
        To = (25.0, 35.0),
        Tc = (35.0, 45.0),
    ),
    stop = :stop,
    optim = (;
        MaxSteps = 5000,
    ),
    metric = :rmse,
)

In [ ]:
p = visualize(corn_obs, :date, :LfNo;
    xlim = (Date(2002, 5), Date(2002, 9)),
    ylim = (0, 20),
    name = "Obs",
)
visualize!(p, LeafAppearanceBF, :t, :n;
    config = (labf_config, labf_calib_config),
    stop = :stop,
    kind = :line,
    name = "Model",
)

In [ ]:
evaluate(LeafAppearanceBF, corn_obs;
    config = (labf_config, labf_calib_config),
    index = :date => :t,
    target = :TotLfNo => :n,
    stop = :stop,
    metric = :rmse,
)

## Chilling-Forcing Model

### Ex 9.3
\In this exercise, we will calibrate and evaluate a two-stage tree phenology model for predicting peak bloom dates of Yoshino cherries in Washington DC. Because of historical, cultural, and economic importance of these Tidal Basin cherry trees, the National Park Service horticulture staff pays meticulous attention to monitor their flowering phenology and bloom dates trees (See how NPS collect cherry phenology data: https://www.nps.gov/subjects/cherryblossom/bloom-watch.htm). That is, data quality is likely to be quite good. Our own UW campus also has many cherry trees and the Yoshino cherries in the Quad in particular are touted for their spectacular blooms and attract many visitors from the public who appreciate the spring blooms. Until recently there was no coordinated effort of monitoring the bloom time of UW cherries so far except for some quasi-observational data based on local and campus news (*e.g.*, Daily), social media (*e.g.*, [UW Cherry Blossom Twitter](https://twitter.com/uwcherryblossom)), and casual records (See Chung et al. (2011)). This means that data quality is likely to be low for UW cherry blossoms. 

The chilling-forcing model (Cesaraccio et al., 2004) has been implemented in the Cropbox framework and will be available for you. The historical peak bloom dates of Washington DC and UW campus will be available (`Cherry_PBD.xls`) along with the corresponding temperature data at nearby weather stations for this exercise.

#### Model
Before we move on, here is a bit explanation of theory behind the model. [Cesaraccio *et al.*](https://doi.org/10.1016/j.agrformet.2004.03.002) split up dormancy break into two distinctive steps, rest and quiescence, that required the same amount of chilling and subsequent anti-chilling accumulation in the opposite direction. Later this model was [extended](https://doi.org/10.1371/journal.pone.0027439) to predict peak bloom date of cherry tree coming after bud burst. The following equation represents our adaptation of this chilling and forcing (CF) model with three parameters: temperature threshold ($T_c$), chilling ($R_{\mathtt{C}}$) and forcing ($R_{\mathtt{F}}$) requirements.

The original model used linear interpolation between maximum and minimum temperature values (Figure 9.4) to incorporate daily fluctuation of temperature. In this exercise, we use a much simpler form of the equation which works better when used with hourly temperature, yet with daily dataset for the sake of simplicity.

$$
\begin{aligned}
T_a & = \begin{cases}
  T & \text{if } T \geq 0 \\
  0 & \text{otherwise}
\end{cases} \\
\Delta T & = \frac{T_a - T_c}{\Delta t} \\
c & = \begin{cases}
  \Delta T & \text{if } \Delta T < 0 \\
  0 & \text{otherwise}
\end{cases} \\
f & = \begin{cases}
  \Delta T & \text{if } \Delta T > 0 \\
  0 & \text{otherwise}
\end{cases} \\
\sum_{t_0}^{\hat{t}_d-1} c & > R_{\mathtt{C}} \geq \sum_{t_0}^{\hat{t}_d} c \\
\sum_{\hat{t}_d}^{\hat{t}-1} f & < R_{\mathtt{F}} \leq \sum_{\hat{t}_d}^{\hat{t}} f \\
\end{aligned}
$$

In [ ]:
@system ChillForceEstimator(Temperature, Controller) begin
    Tc: temperature_threshold ~ preserve(parameter, u"°C")
    Rc: chilling_requirement  ~ preserve(parameter, u"K")
    Rf: forcing_requirement   ~ preserve(parameter, u"K")

    y(t = calendar.last): year              => Dates.year(t)      ~ preserve::int
    doy(t): day_of_year                     => Dates.dayofyear(t) ~ track::int(u"d")
    fdr(year): forced_dormancy_release      => Date(year, 2, 15) ~ preserve::date(parameter)
    

    Ta(T)                                    ~ track(min = 0, u"°C")
    Δt(context.clock.step)                   ~ preserve(u"d")
    ΔT(Ta, Tc, Δt)         => (Ta - Tc) / Δt ~ track(u"K/d")

    c(ΔT): chilling             ~ track(max = 0, u"K/d")
    C(c):  chilling_accumulated ~ accumulate(when = !d, u"K")

   # d(C, Rc):    chilling_done => (C <= Rc) ~ flag
    d(C, Rc, fdr, t): chilling_done => (C <= Rc || fdr <= t) ~ flag    

    f(ΔT): forcing              ~ track(min = 0, u"K/d")
    F(f):  forcing_accumulated  ~ accumulate(when = d, u"K")

    m(F, Rf): match            => (F >= Rf) ~ flag
    stop(m, s = calendar.stop) => (m || s)  ~ flag
end

For your reference, here is our implementation of the original model described by Cesaraccio *et al*. Note that it requires daily maximum and minimum temperature data.

In general, we can find these models similar to the GDD model we made earlier. A notable difference is that now we have two separate variables for accumulating chilling (cumulative `C` with the rate `c`) and forcing (cumulative `F` with the rate `f`). The end of chilling accumulation, which triggers the onset of forcing accumulation, flips `chilling_done` flag (`d`) which is used in the `when` tag for both accumulating variables.

We also need to think about handling exceptions which we would eventually encounter when trying to plug a large range of parameter values for calibration. There is a chance that chilling requirement is never fulfilled by either too high chilling requirement or too low rate of chilling accumulation in this `sequential` modeling approach. To prevent such case, we set`forced_dormancy_release` date to `Feb 15th` of given year. Is it reasonable safety net? 

Alternatively, we may use other ways (e.g., photoperiod, parallel forcing) to ensure forcing period begins regardless of chilling requirement being met instead of using `forced domancy release`. We could also simply run the  model without triggering forcing period. If we go with this approach, the model may never fulfill the forcing requirement within the given time frame which was set by the `last` day of `Calendar`, leaving no result in the output. We take care of such cases by supplying a `callback` function for `simulate()` to fill in *no result* with an arbitrary estimation date (365th day of the year).

In [ ]:
callback(s, m) = if s.stop' && !s.match'
    m.result[end][:doy] = 365u"d"
end

For convenience, we make `estimate()` function to call `simulate()` with some default arguments including `stop` and `callback` as well as configuration generated for each year by `cherry_config()` function explained later.

In [ ]:
estimate(S, years;
    config,
    index = [:year, :doy],
    target = [],
    stop = :stop,
    kwargs...
) = simulate(S;
    configs = @config(config + cherry_config.(years)),
    index,
    target,
    stop,
    snap = stop,
    callback,
    kwargs...
)

#### Weather

In [ ]:
dc_weather = CSV.read("dc_weather.csv", DataFrame) |> unitfy

In [ ]:
visualize(dc_weather, :date, [:Tmin, :Tmax, :Tavg]; kind = :line)

In [ ]:
dc_config = @config (
    ChillForceEstimator => (;
        data = dc_weather,
    ),
)

#### Configuration

For each year, we need to specify start and end date of the simulation which are set by `init` and `last` of `Calendar`. Also don't forget setting daily time step for `Clock`. We create `cherry_config()` function to generate a configuration we need for each year.

In [ ]:
cherry_config(year) = @config (
    Calendar => (;
        init = ZonedDateTime(year-1, 10,  1, tz"UTC"),
        last = ZonedDateTime(year,    5, 31, tz"UTC"),
    ),
    Clock => (;
        step = 1u"d",
    ),
)

In [ ]:
cherry_config(2017)

Then we need another configuration to supply parameters for chilling forcing model. For now, let's have some arbtirary values. We'll recalibrate them later.

In [ ]:
cf_config = @config (
    ChillForceEstimator => (;
        Tc = 10,
        Rc = -100,
        Rf = 100,
    ),
)

#### Test

Here is simulation result for the year of 2017. Note we use `simulate()` with a handmade configuration instead of relying on `estimate()` for testing. Let's run it for about 8 months.

In [ ]:
cf_df = simulate(ChillForceEstimator;
    config = (cf_config, dc_config, cherry_config(2017)),
    index = :t,
    target = [:T, :Ta, :Tx, :Tn, :Tc, :c, :C, :Rc, :f, :F, :Rf, :match, :stop, :doy],
    stop = 8*30u"d",
)

Here is what temperature (`T`) trend looks like during the period.

In [ ]:
visualize(cf_df, :t, :T; #= [:Tx, :Tn];=#
    kind = :line,
    ylim = (-10, 30),
)

Then, here is `Ta` with all negative values stripped out. It is the value going to be used for calculating the degree of chilling and forcing. `Tc` is a threshold parameter used for calculating the degree of chilling and forcing.

In [ ]:
visualize(cf_df, :t, [:Ta, :Tc];
    kind = :line,
    ylim = (-10, 30),
)

Now we have the degree of chilling (`c`) and forcing (`f`) per unit time (day).

In [ ]:
visualize(cf_df, :t, [:c, :f];
    kind = :line,
    ylim = (-20, 20),
)

We can more clearly see how chilling and forcing units respond to the base temperature ($T_c$) thanks to interactive plot.

In [ ]:
manipulate(ChillForceEstimator, :t, [:c, :f];
    config = (cf_config, dc_config, cherry_config(2017)),
    parameters = ChillForceEstimator => :Tc => 0:0.1:20,
    stop = 8*30u"d",
    kind = :line,
    ylim = (-20, 20),
)

The next plot shows how chilling and forcing would accumulate over the period. Note that chilling accumulation (`C`) starts first. Forcing accumulation (`F`) doesn't start until the chilling requirement (`Rc`) is fullfilled. Once the requirement is met, chilling accumulation stops. The output of model, an estimated date of particular phenological event, is determined when forcing accmulation (`F`) make a cross with forcing requirement (`Rf`), which is right before April in this example.

In [ ]:
visualize(cf_df, :t, [:C, :F, :Rc, :Rf];
    kind = :line,
    ylim = (-300, 500),
)

Here is an interactive version.

In [ ]:
manipulate(ChillForceEstimator, :t, [:C, :F, :Rc, :Rf];
    config = (cf_config, dc_config, cherry_config(2017)),
    parameters = ChillForceEstimator => (;
        Tc = 0:0.1:20,
        Rc = -300:0,
        Rf = 0:500,
    ),
    stop = 8*30u"d",
    kind = :line,
    ylim = (-300, 500),
)

Now, let's try `estimate()` to get the answer more directly.

In [ ]:
estimate(ChillForceEstimator, [2017]; config = (cf_config, dc_config))

We may provide additional options as we used to do for `simulate()`.

In [ ]:
estimate(ChillForceEstimator, [2017]; config = (cf_config, dc_config), target = [:C, :F])

A nice feature of `estimate()` is that it supports simulation of multiple years in a convenient way. We'll revisit this later in this notebook.

In [ ]:
estimate(ChillForceEstimator, 2010:2018; config = (cf_config, dc_config), target = [:C, :F])

#### Ex 9.3.1

Using Cropbox, calibrate the chilling-forcing (CF) models using the Washington DC **Yoshino cherry** blooms and Reagan National weather data **from 1991 to 2010**. Provide parameter estimates and model performance measures for calibration (*i.e.*, RMSE, EF). How do the parameter estimates compare with the ones published in Chung et al. (2011)? If different, why do you think there's such difference when used the same data sets for calibration? 

#### Observations
We have a dataset with recordings of peak bloom dates from cherry trees in Washington, DC.

In [ ]:
cherry_dc_obs = CSV.read("cherry_dc_obs.csv", DataFrame)  |> unitfy

In [ ]:
cherry_dc_obs[!,:doy]

#### Calibration

We'll calibrate parameters (`Tc`, `Rc`, `Rf`) using the observation dataset from 1991 to 2010.

In [ ]:
cf_calib_config = calibrate(ChillForceEstimator, cherry_dc_obs;
    configs = @config(dc_config + cherry_config.(1991:2010)),
    index = :year,
    target = :doy,
    parameters = ChillForceEstimator => (;
        Tc = (3, 8),
        Rc = (-300, 0),
        Rf = (0, 300),
    ),
    stop = :stop,
    snap = :stop,
    callback,
    optim = (;
        MaxSteps = 1000,
    ),
)

In [ ]:
# cf_calib_config = @config ChillForceEstimator => (;
#     Tc = 5.94915,
#     Rc = -149.549,
#     Rf = 128.738,
# )

Now we run the model with calibrated parameters.

In [ ]:
est = estimate(ChillForceEstimator, 1991:2010;
    config = (cf_calib_config, dc_config),
    target = [:C, :F],
)

Here are actual dates for the same period.

In [ ]:
cherry_dc_obs[1991 .<= cherry_dc_obs[!, :year] .<= 2010, :]

Then let's create a helper function `compare()` to make a plot for comparing observation and estimation of the model.

In [ ]:
compare(S, obs, years; config, kw...) = begin
    E = estimate(S, years; config)
    O = obs[obs[!, :year] .∈ Ref(years), :]
    p = plot(O, :year, :doy; name = "Obs", kind = :line, kw...)
    plot!(p, E, :year, :doy; name = "Model", kind = :line)
end

Again, here is the result of calibration from 1991 to 2010.

In [ ]:
compare(ChillForceEstimator, cherry_dc_obs, 1991:2010;
    config = (cf_calib_config, dc_config),
    ylim = (75, 110),
)

In [ ]:
evaluate(ChillForceEstimator, cherry_dc_obs;
    configs = @config(cf_calib_config + dc_config + cherry_config.(1991:2010)),
    index = :year,
    target = :doy,
    stop = :stop,
    snap = :stop,
    metric = :rmse,
)

In [ ]:
evaluate(ChillForceEstimator, cherry_dc_obs;
    configs = @config(cf_calib_config + dc_config + cherry_config.(1991:2010)),
    index = :year,
    target = :doy,
    stop = :stop,
    snap = :stop,
    metric = :ef,
)

#### Ex 9.3.2
Evaluate the model performance using the **Yoshino cherry** blooms and Reagan National weather data **from 1937 to 1990**. Provide parameter estimates and model performance measures for calibration (*i.e.*, RMSE, EF). Interpret and discuss the results.

#### Validation

In [ ]:
compare(ChillForceEstimator, cherry_dc_obs, 1937:1990; config=(cf_calib_config, dc_config), ylim=(70,120))

In [ ]:
evaluate(ChillForceEstimator, cherry_dc_obs;
    configs = @config(cf_calib_config + dc_config + cherry_config.(1937:1990)),
    index = :year,
    target = :doy,
    stop = :stop,
    snap = :stop,
    metric = :rmse,
)

In [ ]:
evaluate(ChillForceEstimator, cherry_dc_obs;
    configs = @config(cf_calib_config + dc_config + cherry_config.(1937:1990)),
    index = :year,
    target = :doy,
    stop = :stop,
    snap = :stop,
    metric = :ef,
)

#### Ex 9.3.3

Import Sea-Tac airport weather data from the weather file and UW Yoshino bloom data. Using the parameter estimates from Question 1 above for the DC data, apply the CF models to predict Yoshino cherry bloom dates at UW since 2005 using Sea-Tac airport temperature data. Plot predicted and observed data over the entire period. Evaluate model performance using the appropriate metrics, interpret and discuss the results. What would you do to improve the predictions for UW cherry bloom times?

#### Application to UW cherries

In [ ]:
seatac_weather = CSV.read("seatac_weather.csv", DataFrame) |> unitfy

In [ ]:
visualize(seatac_weather, :date, [:Tmin, :Tmax, :Tavg]; kind = :line)

In [ ]:
uw_config = @config (
    ChillForceEstimator => (;
        data = seatac_weather,
    ),
)

In [ ]:
cherry_uw_obs = CSV.read("cherry_uw_obs.csv", DataFrame) |> unitfy

In [ ]:
compare(ChillForceEstimator, cherry_uw_obs, 2005:2019;
    config = (cf_calib_config, uw_config),
    ylim = (70, 120),
)

In [ ]:
evaluate(ChillForceEstimator, cherry_uw_obs;
    configs = @config(cf_calib_config + uw_config + cherry_config.(2005:2019)),
    index = :year,
    target = :doy,
    stop = :stop,
    snap = :stop,
    metric = :rmse,
)

In [ ]:
evaluate(ChillForceEstimator, cherry_uw_obs;
    configs = @config(cf_calib_config + uw_config + cherry_config.(2005:2019)),
    index = :year,
    target = :doy,
    stop = :stop,
    snap = :stop,
    metric = :ef,
)

It doesn't look too good here. How it would be like if we did calibration? Also note that the original implementation of the model with forced dormancy release seemed to work much better with these new datasets, at least with positive model efficiency (EF) values. What would that suggest?